In [1]:
!pip install vllm==0.6.0
!pip install scikit-learn



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import string
from vllm import LLM, SamplingParams
from tqdm import tqdm
import torch

In [3]:
from datasets import load_dataset

# load files that match the grep pattern
dataset = load_dataset("irene93/news-analysis-ko")
# dataset = dataset['test'][:200]

In [4]:
torch.cuda.empty_cache()

In [5]:
# device = torch.device('cuda:1')

# llm = LLM(model='meta-llama/Meta-Llama-3-8B', gpu_memory_utilization=0.8, device=device)


llm = LLM(model='./model_900')



WARNING 04-03 08:18:11 arg_utils.py:862] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-03 08:18:11 config.py:999] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 04-03 08:18:11 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='./model_900', speculative_config=None, tokenizer='./model_900', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-03 08:18:17 model_runner.py:926] Loading model weights took 14.9888 GB
INFO 04-03 08:18:18 gpu_executor.py:122] # GPU blocks: 28050, # CPU blocks: 2048
INFO 04-03 08:18:20 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-03 08:18:20 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-03 08:18:33 model_runner.py:1335] Graph capturing finished in 13 secs.


In [9]:
testdata = dataset['test']

In [10]:
inputs =  testdata['input'][:200]

In [8]:
# 프롬프트 템플릿 준비
system="당신은 주어진 뉴스를 분석하는 챗봇입니다. **지시사항**:- 주어진 뉴스에 대하여 summary, advr, stk_code, sent_score 분석하고 반드시 json 형태로 출력하세요. - summary는 1~3줄 사이로 작성합니다.- advr는 해당 본문이 광고면 1 광고가 아닐경우에 0 으로 정수 1개의 값으로 출력하세요.- stk_code는 해당 본문에서 언급된 종목명을 찾고, 그 종목명의 종목 코드를 찾아 파이썬 리스트 형태로 작성하세요. - sent_score는 해당 본문이 긍정적일경우 1 부정적일경우 -1 , 긍정적이지도 부정:적이지도 않을경우 0 으로 정수 1개의 값을 출력하세요 - 본문: 이 주어지면 결과: 다음에 json 형태로 작성하세요"


template = string.Template("""<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

\n 뉴스데이터 :${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|> 
""")
predicts =[]
for i, conv in enumerate(tqdm(inputs)):
        
    # 메시지 준비
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": conv}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })
        # 샘플링 파라미터 설정
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    
    # 생성
    outputs = llm.generate([prompt], sampling_params)
    
    # 결과 출력
    for output in outputs:
    
        
        predicts.append(output.outputs[0].text)

          

100%|██████████| 200/200 [07:11<00:00,  2.16s/it]


In [49]:
for output in outputs:
    print(output.outputs[0].text)

{'summary': '구글 클라우드가 한국 공공기관 클라우드 시장에 진출을 위해 CSAP 하 등급 인증을 획득했습니다. 이는 마이크로소프트가 작년 12월에 동일한 인증을 받은 후 두 번째로 발생한 일이며, 아마존 웹서비스의 심사도 진행 중입니다. 이러한 글로벌 클라우드 기업들의 진출으로 공공 클라우드 시장의 경쟁이 더욱 치열해질 것으로 예상됩니다.', 'sktlist': ['구글 클라우드', '마이크로소프트', '아마존웹서비스', '네이버', 'NHN', 'KT클라우드', '한국인터넷진흥원', '과학기술정보통신부'], 'sentiment': '1', 'advr': '0'}
{'summary': '조순열 제98대 서울변호사회 회장 후보는 서부지법 폭력 사태를 법치주의 훼손으로 보고 엄중히 처벌을 촉구했습니다. 그는 헌법과 법률 위반 행위는 용납할 수 없다고 강조하며 사법부에 대한 위협을 경계했습니다. 대법원과 법조 주요 기관도 법치주의에 대한 우려를 표명하며 강력 대응을 예고했습니다.', 'sktlist': ['서울변호사회', '대법원', '대검찰청', '경찰'], 'sentiment': '-1', 'advr': '0'}


In [8]:
predicts[22]

"{'summary': '조순열 서울변호사회 회장 후보는 서부지법 폭력 사태를 비판하며 법치주의 훼손을 엄중히 처벌해야 한다고 강조했습니다. 그는 민주주의와 법치주의 위협에 대해 심각한 우려를 표명하며, 헌법과 법률을 위반한 행위는 용납할 수 없다고 밝혔습니다. 대법원과 대검찰청, 경찰도 법치주의에 대한 부정 행위에 대해 강력히 대응할 것임을 예고했습니다.', 'sktlist': ['서울변호사회', '대법원', '대검찰청', '경찰'], 'sentiment': '-1', 'advr': '0'}"

In [11]:
import ast
# predict_json = [ast.literal_eval(a) for a in predicts]

import json, ast,re
predicts_jsons=[]
for predict_str in tqdm(predicts):
    try:
        predict = ast.literal_eval(predict_str)
        predicts_jsons.append(predict)
    except:
        predict_str = predict_str.replace('\'', '\"')
        summary_pattern = r'"summary"\s*:\s*(.*?)\s*,\s*"sktlist"'

        summary = re.findall(summary_pattern,predict_str)[0][1:-1]
        stklist_pattern = r'"sktlist"\s*:\s*(.*?)\s*,\s*"sentiment"'
        stklist = re.findall(stklist_pattern,predict_str)[0][1:-1]
        
        sentiment_pattern = r'"sentiment"\s*:\s*(.*?)\s*,\s*"advr"'
        sentiment = re.findall(sentiment_pattern,predict_str)[0][1:-1]
        
        match = re.search(r'"advr"\s*:\s*"([^"]+)"', predict_str)
        
        if match:
            advr = match.group(1)
        

        


        predicts_jsons.append({'summary':summary,
                        'stklist':stklist,
                        'sentiment':sentiment,
                        'advr':advr})

100%|██████████| 200/200 [00:00<00:00, 23328.91it/s]


In [12]:
outputs = testdata['output'][:200]
outputs_json = [ast.literal_eval(a) for a in outputs]

In [13]:
outputs_json[0]

{'summary': '상하이종합지수와 선전성분지수가 각각 상승하며 중국 증시는 장기 투자 자금 유입 기대감 속에 급등했습니다. 중국 당국의 중장기 자금 증시 유입 방안 발표가 증시 상승에 긍정적인 영향을 미쳤으며 특히 금융주가 두드러진 성과를 보였습니다. 블랙록 관계자는 올해 중국 A주 시장에 대한 긍정적 전망을 제시하며 글로벌 증시 속 자사 시장의 투자 기회를 강조했습니다.',
 'sktlist': ['국무원 신문판공실',
  '보험사',
  '공모펀드',
  '사회보장기금',
  '양로금',
  '국유 보험사',
  '트럼프 행정부',
  '블랙록'],
 'sentiment': '1',
 'advr': '0'}

In [14]:
summary_predict = [a['summary'] for a in predicts_jsons]
summary_true = [a['summary'] for a in outputs_json]

In [15]:
%pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from rouge_score import rouge_scorer

def calculate_rouge(reference_list, candidate_list):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    
    rouge1_f, rouge2_f, rougeL_f = [], [], []
    
    for ref, cand in zip(reference_list, candidate_list):
        scores = scorer.score(ref, cand)
        rouge1_f.append(scores['rouge1'].fmeasure)
        rouge2_f.append(scores['rouge2'].fmeasure)
        rougeL_f.append(scores['rougeL'].fmeasure)
        
    avg_rouge1 = sum(rouge1_f) / len(rouge1_f)
    avg_rouge2 = sum(rouge2_f) / len(rouge2_f)
    avg_rougeL = sum(rougeL_f) / len(rougeL_f)
    
    return {
        'rouge1': avg_rouge1,
        'rouge2': avg_rouge2,
        'rougeL': avg_rougeL
    }

# Calculate ROUGE scores
rouge_scores = calculate_rouge(summary_true, summary_predict)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': 0.5136144200215815, 'rouge2': 0.28555438955273027, 'rougeL': 0.49258413643758947}


In [17]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference_list, candidate_list):
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    
    for ref, cand in zip(reference_list, candidate_list):
        ref_tokens = ref.split()
        cand_tokens = cand.split()
        score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothie)
        bleu_scores.append(score)
        
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

# Calculate BLEU score
bleu_score = calculate_bleu(summary_true, summary_predict)
print("BLEU Score:", bleu_score)


BLEU Score: 0.10539262953913596


In [19]:
# %pip install bert_score
from bert_score import score

def calculate_bertscore(reference_list, candidate_list):
    P, R, F1 = score(candidate_list, reference_list, lang='ko', verbose=True)
    avg_precision = P.mean().item()
    avg_recall = R.mean().item()
    avg_f1 = F1.mean().item()
    
    return {
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1
    }

# Calculate BERTScore
bertscore = calculate_bertscore(summary_true, summary_predict)
print("BERTScore:", bertscore)


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.13 seconds, 176.68 sentences/sec
BERTScore: {'Precision': 0.8264854550361633, 'Recall': 0.8243753910064697, 'F1': 0.8252658843994141}


In [20]:
sentiment_predict = [a['sentiment'] for a in predicts_jsons]
sentiment_true = [a['sentiment'] for a in outputs_json]

In [28]:
sentiment_true[14]

'-1'

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

# 예측값과 실제값 예시 (실제 데이터로 대체)

# true 리스트가 중첩 리스트인 경우 처리 (첫번째 리스트만 추출)
if isinstance(sentiment_true[0], list):
    true_flat = sentiment_true[0]
else:
    true_flat = sentiment_true

# 정확도 계산
accuracy = accuracy_score(true_flat, sentiment_predict)
print(f"정확도 (Accuracy): {accuracy:.4f}")

# 정밀도(precision), 재현율(recall), F1 계산 (클래스별 및 평균 제공)
precision, recall, f1, support = precision_recall_fscore_support(
    true_flat, sentiment_predict, labels=['1', '0', '-1'], average=None
)

# 클래스별 결과 출력
for label, p, r, f, s in zip(['1', '0', '-1'], precision, recall, f1, support):
    print(f"\n클래스 {label} 결과:")
    print(f"정밀도(Precision): {p:.4f}")
    print(f"재현율(Recall): {r:.4f}")
    print(f"F1-Score: {f:.4f}")
    print(f"Support(샘플 개수): {s}")

# macro 평균 F1-score (모든 클래스 중요도 동일하게 가정)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    true_flat, sentiment_predict, average='macro'
)

print("\nMacro 평균 결과:")
print(f"정밀도(Precision): {precision_macro:.4f}")
print(f"재현율(Recall): {recall_macro:.4f}")
print(f"F1-Score: {f1_macro:.4f}")

# Confusion Matrix 출력
cm = confusion_matrix(true_flat, sentiment_predict, labels=['1', '0', '-1'])
print("\nConfusion Matrix:")
print(cm)

# 전체 리포트 보기 좋게 출력
print("\nClassification Report:")
print(classification_report(true_flat, sentiment_predict, labels=['1', '0', '-1']))


정확도 (Accuracy): 0.7950

클래스 1 결과:
정밀도(Precision): 0.8200
재현율(Recall): 0.7593
F1-Score: 0.7885
Support(샘플 개수): 54

클래스 0 결과:
정밀도(Precision): 0.7184
재현율(Recall): 0.8605
F1-Score: 0.7831
Support(샘플 개수): 86

클래스 -1 결과:
정밀도(Precision): 0.9362
재현율(Recall): 0.7333
F1-Score: 0.8224
Support(샘플 개수): 60

Macro 평균 결과:
정밀도(Precision): 0.8249
재현율(Recall): 0.7844
F1-Score: 0.7980

Confusion Matrix:
[[41 13  0]
 [ 9 74  3]
 [ 0 16 44]]

Classification Report:
              precision    recall  f1-score   support

           1       0.82      0.76      0.79        54
           0       0.72      0.86      0.78        86
          -1       0.94      0.73      0.82        60

    accuracy                           0.80       200
   macro avg       0.82      0.78      0.80       200
weighted avg       0.81      0.80      0.80       200



In [75]:
predicts_jsons[0]

{'summary': '상하이 지수가 1.02% 상승하며 오전장 마감을 기록했으며, 중국 증시는 장기 투자 자금 유입에 따른 긍정적인 반응을 보였습니다. 중국 국무원이 발표한 자금 유입 방안이 확대되면서 보험사, 공모펀드 등 다양한 기관의 증시 투자 비중이 늘어나 기대됩니다. 그러나 대외적 불확실성으로 인한 리스크로 인해 일부 매물도 발생했으며, A주 시장을 긍정적으로 보고 있다고 전문가들은 전망합니다.',
 'sktlist': ['중국 국무원', '보험사', '공모펀드', '사회보장기금', '양로금', '트럼프 행정부'],
 'sentiment': '1',
 'advr': '0'}

In [30]:
stklist_predict = [a['sktlist'] if 'sktlist' in a.keys() else a['stklist'] for a in predicts_jsons]
stklist_true = [a['sktlist'] for a in outputs_json]

In [34]:
for i in range(200):
    predict = stklist_predict[i]
    true = stklist_true[i]
    predict_set = set(predict)
    predict_true = set(true)
    diff = predict_set.difference(predict_true)
    difflist = list(diff)
    for item in difflist:
        if item in inputs[i]:
            stklist_true[i] += item

In [38]:
len(stklist_predict)

200

In [39]:
from typing import List
import numpy as np

def evaluate_lists(list_true: List[List], list_pred: List[List]):
    """
    각 리스트별로 개별 비교하고 평균 메트릭을 반환합니다.
    
    Args:
        list_true: 정답 리스트들의 리스트
        list_pred: 예측 리스트들의 리스트

    Returns:
        dict: 평균 Precision, Recall, F1-Score
    """

    def metrics_per_pair(true_sublist, pred_sublist):
        true_set = set(true_sublist)
        pred_set = set(pred_sublist)

        tp = len(true_set & pred_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (
            (2 * precision * recall) / (precision + recall)
            if (precision + recall) > 0 else 0.0
        )

        accuracy = tp / len(true_set | pred_set) if len(true_set | pred_set) > 0 else 0.0

        return accuracy, precision, recall, f1

    all_metrics = [metrics_per_pair(t, p) for t, p in zip(list_true, list_pred)]

    # 평균 계산
    accuracy_avg = np.mean([m[0] for m in all_metrics])
    precision_avg = np.mean([m[1] for m in all_metrics])
    recall_avg = np.mean([m[2] for m in all_metrics])
    f1_avg = np.mean([m[3] for m in all_metrics])

    return {
        'accuracy': accuracy_avg,
        'precision': precision_avg,
        'recall': recall_avg,
        'f1_score': f1_avg
    }

# 사용 예시


metrics = evaluate_lists(stklist_true, stklist_predict)
print(metrics)


{'accuracy': 0.5562896858212432, 'precision': 0.8109374167544909, 'recall': 0.5767977689563861, 'f1_score': 0.649282135051505}


In [35]:
from typing import List

def calculate_metrics(list1: List[List[str]], list2: List[List[str]]):
    def single_metrics(pred, true):
        pred_set, true_set = set(pred), set(true)

        tp = len(pred_set & true_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)

        precision = tp / (tp + fp) if tp + fp else 0.0
        recall = tp / (tp + fn) if tp + fn else 0.0
        accuracy = tp / len(true_set) if true_set else 0.0
        f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0.0

        return accuracy, precision, recall, f1

    accuracy_total, precision_total, recall_total, f1_total = 0, 0, 0, 0
    num_pairs = min(len(list1), len(list2))

    for i in range(num_pairs):
        accuracy, precision, recall, f1 = single_metrics(list1[i], list2[i])
        accuracy_total += accuracy
        precision_total += precision
        recall_total += recall
        f1_total += f1

    # 평균값 반환
    return {
        "Accuracy": accuracy_total / num_pairs,
        "Precision": precision_total / num_pairs,
        "Recall": recall_total / num_pairs,
        "F1-score": f1_total / num_pairs
    }



metrics = calculate_metrics(stklist_predict, stklist_true)

for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


Accuracy: 0.5768
Precision: 0.8109
Recall: 0.5768
F1-score: 0.6493
